In [6]:
!pip install tensorflow tensorflow-gpu opencv-python matplotlib

In [7]:
!pip list

Package                      Version
---------------------------- ---------
absl-py                      1.3.0
asttokens                    2.2.1
astunparse                   1.6.3
backcall                     0.2.0
cachetools                   5.2.0
certifi                      2022.9.24
charset-normalizer           2.1.1
colorama                     0.4.6
contourpy                    1.0.6
cycler                       0.11.0
debugpy                      1.6.4
decorator                    5.1.1
entrypoints                  0.4
executing                    1.2.0
flatbuffers                  22.11.23
fonttools                    4.38.0
gast                         0.4.0
google-auth                  2.15.0
google-auth-oauthlib         0.4.6
google-pasta                 0.2.0
grpcio                       1.51.1
h5py                         3.7.0
idna                         3.4
importlib-metadata           5.1.0
ipykernel                    6.17.1
ipython                      8.7.0
jedi  

In [8]:
print('setup complete')

setup complete
